In [89]:
# Expectation: you collected the console of your WLC, with scan reports, something like:
#Client Scan Reports 
#  Last Report @: 04/19/2023 10:30:02
#  BSSID        : c4f7.d54b.bc2f
#    Time       : 04/19/2023 10:30:02
#    Channel    : 36
#    RSSI (dBm) : 70
#    SNR  (dB)  : 64
#  BSSID        : 1416.9d29.87cf
#    Time       : 04/19/2023 10:30:02
#    Channel    : 40
#    RSSI (dBm) : 76
#    SNR  (dB)  : 70
#  BSSID        : 00fc.ba97.a1cf
#    Time       : 04/19/2023 10:30:02
#    Channel    : 11
#    RSSI (dBm) : 42
#    SNR  (dB)  : 36
#
# And / or something like:
#WiFi to Cellular Steering : Implemented
#Cellular network type: 4G
#Cellular Signal Strength: Good
#
# Load the usual libraries
import pandas as pd
import numpy as np




In [90]:
# Suppose your scan report file is named scan-report_example.txt in the input folder
f = open("input/scan-report.txt")

In [92]:
import re

# Open the input file
with open("input/scan-report.txt", "r") as input_file:
    # Read the content of the input file
    content = input_file.read()

# Find all the lines containing the keyword BSSID and the BSSID value, only retain the BSSID value
bssid_pattern = r"BSSID+\s+\:+\s+([a-fA-F\d]{4}\.[a-fA-F\d]{4}\.[a-fA-F\d]{4})"
bssid_values = re.findall(bssid_pattern, content)

# Find all the lines containing the keyword Channel and the channel value, only retain the channel value up to ch 233 in 6E
channel_pattern = r"Channel+\s+\:+\s+(\d{1,3}|233)"
channel_values = re.findall(channel_pattern, content)

# Find all the lines containing the keyword RSSI and the RSSI value, only retain the RSSI value
rssi_pattern = r"RSSI \(dBm\)\s+:\s+(\d{1,2}|100)"
rssi_values = re.findall(rssi_pattern, content)

# Create a list of tuples containing the values found
data = list(zip(bssid_values, channel_values, rssi_values))

# Write the data to the output file, we create the output file in case you want to use it elsewhere
with open("rawdata.csv", "w") as output_file:
    for row in data:
        # Combine the values with a coma separator
        line = f"{row[0]},{row[1]},{row[2]}\n"
        output_file.write(line)

In [93]:
# Read in the rawdata CSV file
df = pd.read_csv('rawdata.csv', header=None, names=['BSSID', 'channel', 'RSSI'])

# Create a dictionary where each key is a unique BSSID string, and the value is a list of RSSI values
bssid_dict = {}
for index, row in df.iterrows():
    bssid = row['BSSID']
    negative_rssi = -1 * row['RSSI']
    if bssid in bssid_dict:
        bssid_dict[bssid].append(negative_rssi)
    else:
        bssid_dict[bssid] = [negative_rssi]

# Create a new DataFrame from the dictionary
new_df = pd.DataFrame.from_dict(bssid_dict)
new_df.to_csv('output.csv')

In [103]:
# Now each scan report you took was at some location, the raw data is now indexed so that each column is the AP BSSID, and each row the RSSI at each location, from 1 to the end
# Your next job is to create a file with these locaitons, it can be a csv file called locations.csv, where each location has an X and Y value. 
# X is the distance from the left side wall, Y is the distance from the top side wall, so (12, 17) would be, starting from the top left corner of your floor, 12 (units, 
# you may work in meters or in feet) to the right, then 17 units down.
df = pd.read_csv('input/locations.csv', header=0, names=['index', 'X', 'Y'])

# We then add these X Y values to the RSSI data
new_df = pd.concat([df[["X", "Y"]], new_df], axis=1)


In [126]:
# Now we take care of the cellular readings. The phone reports what you would see on the screen, ie. 5G, 4G, 3G or 2G, then a number of bars that 
# represent the quality of the signal, Excellent, Great, Good, Moderate, Poor. I unilaterally (and artificially) decided to associate a scale to these values,
# where poor 5G is better than excellent 4G, and that's because, in my experience, the phone switches to the lower technology ony when the higher technology 
# is not available. Feel free to change the map as you see fit.
# Open the input file, I suppose that it is the same as the scan file (I reload it in case you run this part independently), change if needed
with open("input/scan-report.txt", "r") as input_file:
    # Read the content of the input file
    content = input_file.read()

# Find all the lines containing the keyword Cellular network type, retain the type
cellulartype_pattern = r"Cellular+\s+network+\s+type:+\s+(\d+G)"
cellulartype_values = re.findall(cellulartype_pattern, content)

# Find all the lines containing the keyword Cellular Signal Strength, retain the value/string
strength_pattern = r"Cellular+\s+Signal+\s+Strength:+\s+(Excellent|Great|Good|Moderate|Poor)"
strength_values = re.findall(strength_pattern, content)

# Create a list of tuples containing the values found
data = list(zip(cellulartype_values, strength_values))

# We create a function that maps the values to numbers, as explained at the beginning of this block
def calculate_score(input_list):
    score_list = []
    for item in input_list:
        network = item[0]
        signal = item[1]
        network_score = {'5G': 25, '4G': 40, '3G': 55, '2G': 70}[network]
        signal_score = {'Excellent': 0, 'Great': 3, 'Good': 6, 'Moderate': 9, 'Poor': 12}[signal]
        total_score = network_score + signal_score
        score_list.append(-total_score)
    return score_list

# Then we calculate the scores
output_list = calculate_score(data)
df = pd.DataFrame({'Cellular': output_list})

# We add the scores to the dataframe created above, and we generate the input file

new_df = pd.concat([new_df, df[["Cellular"]]], axis=1)
new_df.to_csv('input/input.csv')